# SRU-Abfrage einer Liste von Identifikatoren + Auslesen von MARC-(Unter)-Feldern

Eine hochgeladene Excel-Liste mit IDs (MMS Id, ISBNA, ISBN, OCLC number, ISSN, ISSNA oder 035 Field) wird ausgelesen und die zugehörigen Titelaufnahmen werden via SRU abgefragt. Danach liest das Script die vorgegebenen Kontroll- und Unterelder aus dem zurückgegebenen MARCXML aus und speichert das Ergebnis als CSV (lokal in den Downloads).

Achtung! Nach Programmstart muss über den Button "Dateien auswählen" (unterhalb des Python-Scrips) die Liste mit den zur Abfrage passenden IDs ausgewählt werden (Spaltenname beachten!).  Beim ersten Mal muss der Zugriff auf Google-Drive zugelassen werden. 
Wenn eine leeres CSV ausgegeben wird, einfach nochmals laufen lassen. 

In [2]:
# Import benötigter Bibliotheken / Module

import urllib.request
import pandas as pd
import time
from bs4 import BeautifulSoup
import csv

# Nur für GoogleColab-GoogleDrive, kann sonst mit # deaktiviert werden.
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

# Damit kann die Liste mit ID's von einem lokalen Laufwerk hochgeladen werden. Namen der Eingabedatei und Spaltennamen anpassen.

uploaded = files.upload()

df = pd.read_excel ('IDs_in.xlsx')
id_list = df['id_035'].tolist()

# SRU-Abfrage: Katalog auswählen (NZ/IZ) und ggf. query anpassen auf alma.mms_id==, alma.other_system_number==, etc.
# !Achtung! MMSID mit Endung 5501 in NZ suchen, mit 5511 in IZ UBE.

#sru_base = 'https://slsp-network.alma.exlibrisgroup.com/view/sru/41SLSP_NETWORK?version=1.2&operation=searchRetrieve&recordSchema=marcxml&query=alma.mms_id==' #für NZ
#sru_base = 'https://slsp-ube.alma.exlibrisgroup.com/view/sru/41SLSP_UBE?version=1.2&operation=searchRetrieve&recordSchema=marcxml&query=alma.mms_id==' #für IZ UBE
sru_base = 'https://slsp-ube.alma.exlibrisgroup.com/view/sru/41SLSP_UBE?version=1.2&operation=searchRetrieve&recordSchema=marcxml&query=alma.other_system_number_active_035==' #für IZ UBE

# Hier den Namen der Ausgabedatei (output) bestimmen, welche auf Google Drive gespeichert wird:

output = "output_sru_035-ids.csv"

f = csv.writer(open(output, "w", encoding="UTF-8", newline=''))
files.download(output)

# Hier können die Spaltenüberschriften der Ausgabedatei bestimmt werden. z.B. f.writerow(["001", "245 a", "655 a", "AVA 8", "AVA d", "AVA j"]) oder stattdessen:

f.writerow(["MMSID", "Titel", "Materialart", "HOL#", "Signatur", "Standort"])

# Das folgende Script liest zwei Arten von MARCXML-Feldern aus:
# 1. Unwiederholbare Felder ohne Unterfeld (z.B. 001 mit MMS-ID).
# 2. Felder, die mehrmals vorkommen können. Die Unterfelder sind aber pro Feld einmalig.
# Mit diesem Script sind also nicht alle Arten von Feldern abgedeckt. Mit weiteren Funktionen können diese aber ergänzt werden.

def ctrlf(field_tag):
    try:
        controlfield = soup.find(tag=field_tag).get_text()
        return(controlfield)
    except:
        pass

def subf(field_tag, subfield_code):
    try:
        fieldname_list = [subfield.find(code=subfield_code).get_text() 
                         for subfield in soup.find_all(tag=field_tag)]
        beautiful_string = "|".join(fieldname_list).replace(";", "/")
        return beautiful_string
    except:
        pass

# Hier beginnt die eigentliche Programmschleife, welche die einzelnen IDs aus der Excelliste liest, laufend zum SRU-String zusammengefügt und diesen abfragt.

for cur_id in id_list:
    sru_strings= sru_base + str(cur_id)
    sru_data = urllib.request.urlopen(sru_strings).read()
   
    #time.sleep(2)   # Damit die SRU-Schnittselle nicht überfodert wird, können die Abfragen optional im 2-Sekunden-Takt ausgeführt werden. 

# BeautifulSoup wird für das Auslesen des XML benötigt.

    soup = BeautifulSoup(sru_data, 'xml')

# Hier werden die MARC-Felder bestimmt, die in die CSV-Datei ausglesen werden sollen. Wichtig ist, dass die MARC-Felder mit den oben definierten Spaltennamen korrrelieren.

    f.writerow([ ctrlf("001"),  subf("245", "a"), subf("655", "a"), 
                subf("AVA", "8"), subf("AVA", "d"), subf("AVA", "j")])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving IDs_in.xlsx to IDs_in (1).xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>